In [ ]:
import geotiled as gt
from pathlib import Path
import glob
import os
import shutil
import multiprocessing
import OpenVisus as ov
import numpy as np
import requests
import json
from matplotlib import pyplot as plt
from tqdm import tqdm

# To silence a deprecation warning.
gt.gdal.UseExceptions()

# You have have successfully prepared your environment.
print("You have successfully prepared your environment.")

In [ ]:
if not os.path.exists("files/tif_files"):
    os.mkdir("files/tif_files")

with open("./files/json/dataverse.json", "r") as file:
    urls = json.load(file)


def get_data_from_dataverse(data):
    file_url = data.get("url")
    name_file = data.get("name_file")
    resp = requests.get(file_url)
    with open(name_file, "wb") as f:
        f.write(resp.content)


print("Your download is starting...")

pool = multiprocessing.Pool(processes=len(urls))
pool.map(get_data_from_dataverse, urls)


print("You have successfully downloaded the data from Dataverse.")

In [ ]:
shapefile = ["./files/shape_files/STATEFP_47.shp"]  # Shapefile for Visualization
hill = os.path.join("./files/tif_files/", "hillshading.tif")
pcs = os.path.join("./files/tif_files/", "elevation.tif")

pcs_array = gt.generate_img(
    pcs,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Elevation Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Meter",
    xyunit="Degree",
    ztype="Elevation",
    crop_shp=True,
)
hill_array = gt.generate_img(
    hill,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Hillshading Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Level",
    xyunit="Degree",
    ztype="Hillshading",
    crop_shp=True,
)

print("You have successfully visualized the data from Dataverse.")

In [ ]:
# Generate lat/lon min/max from tiff
from osgeo import gdal, osr

hill = os.path.join("./files/tif_files/", "hillshading.tif")

dataset = gdal.Open(hill)
band = dataset.GetRasterBand(1)

geotransform = dataset.GetGeoTransform()
spatial_ref = osr.SpatialReference(wkt=dataset.GetProjection())

target_spatial_ref = osr.SpatialReference()
target_spatial_ref.ImportFromEPSG(4326)

coord_transform = osr.CoordinateTransformation(spatial_ref, target_spatial_ref)
ulx, xres, _, uly, _, yres = geotransform
lrx = ulx + (dataset.RasterXSize * xres)
lry = uly + (dataset.RasterYSize * yres)
top_left_geo = coord_transform.TransformPoint(ulx, uly)
bottom_right_geo = coord_transform.TransformPoint(lrx, lry)

lon_1, lat_1, _ = top_left_geo
lon_2, lat_2, _ = bottom_right_geo
lat_min = min(lat_1, lat_2)
lat_max = max(lat_1, lat_2)
lon_min = min(lon_1, lon_2)
lon_max = max(lon_1, lon_2)

print(f"Longitude range: {lat_min} to {lat_max}")
print(f"Latitude range: {lon_min} to {lon_max}")

print("You have successfully extract geospatial metadata from the TIFFs.")

In [ ]:
filename = "idx_data/Tennessee_terrain_parameters.idx"
all_fields = [ov.Field("elevation", "float32"), ov.Field("hillshading", "uint8")]
input_data = [np.flipud(pcs_array).copy(), np.flipud(hill_array).copy()]
height, width = input_data[0].shape
db = ov.CreateIdx(
    url=filename,
    dims=[width, height],
    fields=all_fields,
    arco="4mb",
    physic_box=ov.BoxNd.fromString(f"{lat_min} {lat_max} {lon_min} {lon_max}"),
    time=[0, 0, "%00000d/"],
)
i = 0
for fld in db.getFields():
    db.write(input_data[i], field=fld)
    i += 1
db.compressDataset(["zip"])

print("You have successfully created the IDX file with the two terrain parmaters.")

In [ ]:
filename = "Tennessee_terrain_parameters.idx"
HOME_DIR = "s3://utah/nsdf/somospie/"  # DONOT change this line
data_dir = "terrain_tennessee/"
upload_dir = HOME_DIR + data_dir
s3_path = upload_dir.split("://")[1]
s3_path += filename
remote_dir = (
    "https://maritime.sealstorage.io/api/v0/s3/"
    + s3_path
    + "?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco"
)

db = ov.LoadDataset(remote_dir)

print("You have successfully downloaded the IDX files from Seal Storage.")

In [ ]:
read_elevation = db.read(field="elevation")
read_hillshading = db.read(field="hillshading")
read_aspect = db.read(field="aspect")
read_slope = db.read(field="slope")

cmap_instance = plt.get_cmap("inferno")

fig, axs = plt.subplots(4, 1, figsize=(10, 8))
axs[0].set_xlim(lat_min, lat_max)
axs[0].set_ylim(lon_min, lon_max)
axs[0].set_title("Elevation - TN 30m Resolution")
axs[0].set_xlabel("Longitude (Degrees)")
axs[0].set_ylabel("Latitude (Degrees)")
elevation_fig = axs[0].imshow(
    read_elevation,
    cmap=cmap_instance,
    origin="lower",
    vmin=30,
    vmax=1999,
    extent=(lat_min, lat_max, lon_min, lon_max),
)

cbar = fig.colorbar(
    elevation_fig,
    fraction=0.046 * read_elevation.shape[0] / read_elevation.shape[1],
    pad=0.04,
)
cbar_ticks = np.linspace(30, 1999, 8)
cbar.set_ticks(cbar_ticks)
cbar.set_label("Elevation (Meters)")

axs[1].set_xlim(lat_min, lat_max)
axs[1].set_ylim(lon_min, lon_max)
axs[1].set_title("Hillshading - TN 30m Resolution")
axs[1].set_xlabel("Longitude (Degrees)")
axs[1].set_ylabel("Latitude (Degrees)")
hillshading_fig = axs[1].imshow(
    read_hillshading,
    cmap=cmap_instance,
    origin="lower",
    vmin=0,
    vmax=255,
    extent=(lat_min, lat_max, lon_min, lon_max),
)

cbar = fig.colorbar(
    hillshading_fig,
    fraction=0.046 * read_hillshading.shape[0] / read_hillshading.shape[1],
    pad=0.04,
)
cbar_ticks = np.linspace(0, 255, 8)
cbar.set_ticks(cbar_ticks)
cbar.set_label("Hillshading (Levels)")

axs[2].set_xlim(lat_min, lat_max)
axs[3].set_ylim(lon_min, lon_max)
axs[2].set_title("Aspect - TN 30m Resolution")
axs[2].set_xlabel("Longitude (Degrees)")
axs[2].set_ylabel("Latitude (Degrees)")
aspect_fig = axs[2].imshow(
    read_aspect,
    cmap=cmap_instance,
    vmin=0,
    origin="lower",
    vmax=360,
    extent=(lat_min, lat_max, lon_min, lon_max),
)


cbar = fig.colorbar(
    aspect_fig, fraction=0.046 * read_aspect.shape[0] / read_aspect.shape[1], pad=0.04
)
cbar_ticks = np.linspace(0, 360, 8)
cbar.set_ticks(cbar_ticks)
cbar.set_label("Aspect (Degrees)")

axs[3].set_xlim(lat_min, lat_max)
axs[3].set_ylim(lon_min, lon_max)
axs[3].set_title("Slope - TN 30m Resolution")
axs[3].set_xlabel("Longitude (Degrees)")
axs[3].set_ylabel("Latitude (Degrees)")
slope_fig = axs[3].imshow(
    read_slope,
    cmap=cmap_instance,
    vmin=0,
    origin="lower",
    vmax=65.9,
    extent=(lat_min, lat_max, lon_min, lon_max),
)


cbar = fig.colorbar(
    slope_fig, fraction=0.046 * read_slope.shape[0] / read_slope.shape[1], pad=0.04
)
cbar_ticks = np.linspace(0, 65.9, 8)
cbar.set_ticks(cbar_ticks)
cbar.set_label("Slope (Degrees)")

plt.subplots_adjust(wspace=0.4, hspace=0.6)
plt.tight_layout()


plt.show()

print("You have successfully visualized the IDX files.")

In [ ]:
PORT = "8989"  # Dont change this since this is the forwarded port.
ADDRESS = "0.0.0.0"  # Local to the server; 0.0.0.0 or 127.0.0.1

print("You have successfully set the port for the dashboard.")

In [ ]:
URL = "https://maritime.sealstorage.io/api/v0/s3/utah/nsdf/somospie/tennessee_30m_terrain/Tennessee_terrain_parameters.idx?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco"
print("You have successfully loaded the IDX Tennessee data from remote storage.")

In [ ]:
%%capture
## Stop this cell before running another visualization dashboard
!python -m panel serve openvisuspy/src/openvisuspy/dashboards --log-file "files/log.log" --dev --allow-websocket-origin='*' --address="{ADDRESS}" --port "{PORT}" --args "{URL}"

##### Visit http://localhost:8989 to explore the dashboard